In [2]:
#import important package first 
import numpy as np
from numpy import random as rd
import pandas as pd

In [7]:
#The first step in data cleaning 
#    and variable reduction is to remove variables with extensive number of missing. 
# Clean data step:
# 1.Remove variables with more than 70% missing. 
# 2.Remove non-numerical variables.
# 3.For the remaining variables, impute missing values with median of that variable.

data1 = pd.read_csv('compustat_annual_2000_2017_with link information.csv')


/Users/broteinlee/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,17,33,37,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
#1. Remove variables with more than 70% missing
data2 = data1.loc[:, data1.isnull().mean() < .7]


#2. Remove non-numerical variables
data3 = data2._get_numeric_data()

#3-1.cap and floor outliers at 0.1% and 99.9%.
low = .001
high = .999
quant_df = data3.quantile([low, high])

#filtering values based on computed percentiles. To do that I use an apply by columns
filt_df = data3.apply(lambda x: x[(x>quant_df.loc[low,x.name]) & 
                                    (x < quant_df.loc[high,x.name])], axis=0)

#3-2.For the remaining variables, impute missing values with median of that variable.

data4 = filt_df.fillna(filt_df.median())

print(data4)


          GVKEY  LPERMNO  LPERMCO      LINKDT    datadate   fyear    ajex  \
0       30095.0  54594.0  20000.0  19720424.0  20000531.0  2007.0  1.0000   
1       30095.0  54594.0  20000.0  19720424.0  20010531.0  2000.0  1.0000   
2       30095.0  54594.0  20000.0  19720424.0  20020531.0  2001.0  1.0000   
3       30095.0  54594.0  20000.0  19720424.0  20030531.0  2002.0  1.0000   
4       30095.0  54594.0  20000.0  19720424.0  20040531.0  2003.0  1.0000   
5       30095.0  54594.0  20000.0  19720424.0  20050531.0  2004.0  1.0000   
6       30095.0  54594.0  20000.0  19720424.0  20060531.0  2005.0  1.0000   
7       30095.0  54594.0  20000.0  19720424.0  20070531.0  2006.0  1.0000   
8       30095.0  54594.0  20000.0  19720424.0  20080531.0  2007.0  1.0000   
9       30095.0  54594.0  20000.0  19720424.0  20090531.0  2008.0  1.0000   
10      30095.0  54594.0  20000.0  19720424.0  20100531.0  2009.0  1.0000   
11      30095.0  54594.0  20000.0  19720424.0  20110531.0  2010.0  1.0000   

In [34]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

X = data4.loc[:, data4.columns != 'oiadp']
y = data4['oiadp']

model = stepwise_selection(X, y)

print('resulting features:')
print(model)




/Users/broteinlee/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1574: RuntimeWarning: invalid value encountered in greater
  return (S > tol).sum(axis=-1)
/Users/broteinlee/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/broteinlee/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/broteinlee/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


Add  siv                            with p-value 0.0


LinAlgError: SVD did not converge